In [ ]:
from transformers import pipeline

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

ask_llm = pipeline(
    model= model_name,
    device="cuda"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cuda


In [ ]:
from datasets import load_dataset

raw_data = load_dataset("json", data_files="RPG_Game_DataSet.jsonl")
raw_data

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['system', 'input', 'output'],
        num_rows: 36000
    })
})

In [ ]:
raw_data["train"][0]

{'system': 'This is a task you must complete by returning only the output.\nDo not include explanations, code, or extra text—only the result.\n',
 'input': 'create_next_level\n\nself = LevelManager(current_level=2, current_difficulty=1, prev_levels=[], prev_level_maps=[])\nlast_levels = []\ndifficulty = 1',
 'output': '{"name": 3, "difficulty": 1, "time": 300, "width": 10, "height": 10, "num_wall": 5, "num_enemies": 2}'}

## Tokenization


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_name
)

def preprocess(sample):
    text = sample["system"] + "\n" + sample["input"] + "\n" + sample["output"]

    tokenized = tokenizer(
        text,
        max_length=4096,
        truncation=True,
        padding="max_length",
    )

    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

data = raw_data.map(preprocess, remove_columns=raw_data["train"].column_names)

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]



### Preview Tokenized Sample

In [ ]:
print(data["train"][0])

{'input_ids': [1, 910, 338, 263, 3414, 366, 1818, 4866, 491, 7863, 871, 278, 1962, 29889, 13, 6132, 451, 3160, 7309, 800, 29892, 775, 29892, 470, 4805, 1426, 30003, 6194, 278, 1121, 29889, 13, 13, 3258, 29918, 4622, 29918, 5563, 13, 13, 1311, 353, 21597, 3260, 29898, 3784, 29918, 5563, 29922, 29906, 29892, 1857, 29918, 12765, 3953, 29891, 29922, 29896, 29892, 12379, 29918, 5563, 29879, 11759, 1402, 12379, 29918, 5563, 29918, 10339, 11759, 2314, 13, 4230, 29918, 5563, 29879, 353, 5159, 13, 12765, 3953, 29891, 353, 29871, 29896, 13, 6377, 978, 1115, 29871, 29941, 29892, 376, 12765, 3953, 29891, 1115, 29871, 29896, 29892, 376, 2230, 1115, 29871, 29941, 29900, 29900, 29892, 376, 2103, 1115, 29871, 29896, 29900, 29892, 376, 3545, 1115, 29871, 29896, 29900, 29892, 376, 1949, 29918, 11358, 1115, 29871, 29945, 29892, 376, 1949, 29918, 264, 331, 583, 1115, 29871, 29906, 29913, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

## LoRA


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = "cuda",
    torch_dtype = torch.float16
)

lora_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    target_modules = ["q_proj", "k_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)

## Training / Fine Tuning


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./tinyllama-finetune",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    learning_rate=0.001,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=10,         # log every 10 steps
    report_to="none",         # disable WandB unless you want it
    save_strategy="epoch",    # save model each epoch
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data["train"]
)

trainer.train()

Step,Training Loss
10,2.260500
20,0.479900
30,0.336600
40,0.247600
50,0.199300
60,0.197900
70,0.176800
80,0.136400
90,0.103900
100,0.096400


Step,Training Loss
10,2.260500
20,0.479900
30,0.336600
40,0.247600
50,0.199300
60,0.197900
70,0.176800
80,0.136400
90,0.103900
100,0.096400


## Test Fine-Tuned Model


In [ ]:
model_path = "./finetuned-tinyllama"

# Save model and tokenizer
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)
